In [1]:
import os

import numpy as np
from astropy.coordinates import Angle

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH

In [2]:
testfile = os.path.join(DATA_PATH, 'zen.2458661.23480.HH.uvh5')
uv_raw = UVData()
uv_raw.read_uvh5(testfile)

uv_phase = uv_raw.copy()

In [3]:
uv_phase.reorder_blts(order="time", minor_order="baseline")
uv_raw.reorder_blts(order="time", minor_order="baseline")
uv_phase.phase(ra=0, dec=0, epoch="J2000")
times = np.unique(uv_phase.time_array)
time_set_1, time_set_2 = times[::2], times[1::2]

uv_phase_1 = uv_phase.select(times=time_set_1, inplace=False)
uv_phase_2 = uv_phase.select(times=time_set_2, inplace=False)

uv_raw_1 = uv_raw.select(times=time_set_1, inplace=False)
uv_raw_2 = uv_raw.select(times=time_set_2, inplace=False)

uv_raw_1copy = uv_raw_1.copy()


In [4]:
phase_center_radec = (Angle('0d').rad, Angle('-30d').rad)

# phase each half to different spots
uv_raw_1.phase(ra=0,
               dec=0,
               epoch="J2000"
               )
uv_raw_2.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

# phase original to phase_center_radec
uv_raw.phase(ra=phase_center_radec[0],
             dec=phase_center_radec[1],
             epoch="J2000"
             )


In [5]:
uv_raw_1copy2 = uv_raw_1.copy()
uv_raw_1copy2.unphase_to_drift()

# check that unphasing gets us back to what we started with
vis_diff = uv_raw_1copy.data_array - uv_raw_1copy2.data_array
print(np.max(np.abs(vis_diff)))

assert uv_raw_1copy == uv_raw_1copy2

# phase each the same way
uv_raw_1copy2.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

uv_raw_1copy.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

vis_diff = uv_raw_1copy.data_array - uv_raw_1copy2.data_array
print(np.max(np.abs(vis_diff)))

assert uv_raw_1copy2 == uv_raw_1copy

3.253907165344266e-11
7.587781535024039
data_array parameter value is array, values are not close
parameter _data_array does not match. Left is [[[[ 1.31315800e+06-4.92607950e+00j  1.22309000e+06-4.58820536e+00j]
   [ 1.34179400e+06-5.04659767e+00j  1.24748500e+06-4.69189376e+00j]
   [ 1.37036200e+06-5.16741837e+00j  1.27538500e+06-4.80927513e+00j]
   [ 1.39908300e+06-5.28937513e+00j  1.29928800e+06-4.91209001e+00j]]]


 [[[-1.51469757e+04-1.55575386e+05j -5.85161523e+03-3.28987024e+04j]
   [-8.49061126e+03-1.48321012e+05j -2.26460359e+03-3.60374010e+04j]
   [-3.69635167e+03-1.54607746e+05j -4.43773136e+03-3.56183150e+04j]
   [ 2.45079220e+03-1.55582947e+05j -2.56264700e+03-3.74589917e+04j]]]


 [[[-1.20410193e+03+6.96503077e+02j -5.25750687e+03+2.09432267e+04j]
   [-9.44779674e+02+1.76375320e+03j -1.95752336e+03+1.98660530e+04j]
   [ 8.56345310e+02-1.09131330e+03j -7.37937279e+03+2.30081177e+04j]
   [-4.96394864e+02-1.50039866e+03j -7.67874203e+03+1.76092107e+04j]]]


 [[[-4.84628387e

AssertionError: 

In [ ]:
uv_out = uv_raw_1.fast_concat(uv_raw_2, axis='blt', phase_center_radec=phase_center_radec)
uv_out.reorder_blts(order='time', minor_order='baseline')

In [ ]:
assert uv_out.phase_type == "phased"
uv_out.history = uv_raw.history
assert (uv_out.phase_center_ra, uv_out.phase_center_dec) == phase_center_radec
assert uv_out == uv_raw
